# Visualize taxonomy and alpha/beta diversities

## Platform dependent part
- Resolve platform setup
- the difference to local imports should be resolved by setting the Blue Cloud VRE well, Colab will still be an issue.

In [ ]:
import sys
import os
import logging
from IPython import get_ipython
logger = logging.getLogger(name="Diversity analysis app")

if 'google.colab' in str(get_ipython()):
    print('Setting Google colab, you will need a ngrok account to make the dashboard display over the tunnel. \
    https://ngrok.com/')
    # clone the momics-demos repository to use it to load data
    try:
        os.system('git clone https://github.com/palec87/momics-demos.git')
        logger.info(f"Repository cloned")
    except OSError as e:
        logger.info(f"An error occurred while cloning the repository: {e}")

    sys.path.insert(0,'/content/momics-demos')

    # this step takes time beacause of many dependencies
    os.system('pip install marine-omics')

from momics.utils import (
    memory_load, reconfig_logger,
    init_setup, get_notebook_environment,
)

# Set up logging
reconfig_logger()

# Determine the notebook environment
env = get_notebook_environment()

init_setup()
logger.info(f"Environment: {env}")

## Imports

In [ ]:
import warnings
import holoviews as hv

from functools import partial
warnings.filterwarnings('ignore')

import pandas as pd
import panel as pn

from mgo.udal import UDAL

# All low level functions are imported from the momics package
from momics.loader import load_parquets_udal
from momics.metadata import get_metadata_udal, enhance_metadata
import momics.plotting as pl
from momics.panel_utils import (
    diversity_select_widgets, create_indicators_diversity,
    serve_app, close_server,
)

## User settings

In [ ]:
DEBUG = True  # enable stdout logging

## Loading

In [ ]:
udal = UDAL()

In [ ]:
# parquet files
if 'google.colab' in str(get_ipython()):
    root_folder = os.path.abspath(os.path.join('/content/momics-demos'))
else:
    root_folder = os.path.abspath(os.path.join('../'))

assets_folder = os.path.join(root_folder, 'assets')

In [ ]:
@pn.cache()
def get_data():
    return load_parquets_udal()

# Load and merge metadata
@pn.cache()
def get_full_metadata():
    return get_metadata_udal()

@pn.cache()
def get_valid_samples():
    df_valid = pd.read_csv(
        os.path.join(root_folder, 'data/shipment_b1b2_181.csv')
    )
    return df_valid

In [ ]:
# Load metadata
full_metadata = get_full_metadata()

# filter the metadata only for valid 181 samples
valid_samples = get_valid_samples()
full_metadata = enhance_metadata(full_metadata, valid_samples)

# LOADing data
mgf_parquet_dfs = get_data()

In [ ]:
# select categorical columns from metadata
categorical_columns = sorted(full_metadata.select_dtypes(include=['object', "boolean"]).columns)
cat_to_remove = ["ref_code", "samp_description", "source_mat_id", "source_mat_id_orig",
                 'ENA_accession_number_sample',
]
# remove columns that are not needed for the analysis
categorical_columns = [k for k in categorical_columns if k not in cat_to_remove]

# select numerical columns from metadata
numerical_columns = sorted(full_metadata.select_dtypes(include=['int64', 'float64']).columns)
numerical_columns.remove("chem_administration")
numerical_columns.remove("bac_prod")
numerical_columns.remove("bac_prod_method")
numerical_columns.remove("biomass")
numerical_columns.remove("biomass_method")
numerical_columns.remove("diss_carb_dioxide")
numerical_columns.remove("diss_org_carb")
numerical_columns.remove("diss_org_carb_method")
numerical_columns.remove("diss_inorg_carb")
numerical_columns.remove("diss_inorg_carb_method")
numerical_columns.remove("diss_org_nitro")
numerical_columns.remove("diss_carb_dioxide_method")
numerical_columns.remove("diss_org_nitro_method")
numerical_columns.remove("down_par")
numerical_columns.remove("down_par_method")
numerical_columns.remove("long_store")
numerical_columns.remove("membr_cut")
numerical_columns.remove("n_alkanes")
numerical_columns.remove("n_alkanes_method")
numerical_columns.remove("part_org_carb")
numerical_columns.remove("part_org_carb_method")
numerical_columns.remove("part_org_nitro")
numerical_columns.remove("part_org_nitro_method")
numerical_columns.remove("petroleum_hydrocarb")
numerical_columns.remove("petroleum_hydrocarb_method")
numerical_columns.remove("sulfate")
numerical_columns.remove("sulfate_method")
numerical_columns.remove("sulfide")
numerical_columns.remove("sulfide_method")
numerical_columns.remove("water_current")
numerical_columns.remove("water_current_method")

# assert len(full_metadata.columns) == len(numerical_columns) + len(categorical_columns) + len(cat_to_remove)  # + for removed cats

if DEBUG:
    logger.info(f"Data table names are:\n{mgf_parquet_dfs.keys()}")
    logger.info(f"Categorical metadata columns are:\n{categorical_columns}")
    logger.info(f"Numerical metadata columns are:\n{numerical_columns}")

In [ ]:
df = mgf_parquet_dfs['ssu'].copy()
if DEBUG:
    logger.info(f'Number of unique ref_codes: {df.ref_code.nunique()}')

In [ ]:
(select_table, select_cat_factor, 
 select_table_beta, select_taxon,
 select_beta_factor, beta_norm,
 ) = diversity_select_widgets(categorical_columns, numerical_columns)

## Alpha diversity

In [ ]:
pn.extension("tabulator")
hv.extension("bokeh", "plotly")
if 'google.colab' in str(get_ipython()):
    pn.extension(comms='colab')

In [ ]:
sort_alpha = pn.widgets.RadioBoxGroup(
    name='Sort by',
    options=['factor', 'values'],
    inline=True,
)
sort_alpha.value = 'factor'

backend = pn.widgets.RadioBoxGroup(
    name='Backend',
    options=['matplotlib', 'hvplot'],
    inline=True,
)

pn.Column(
    pn.Row(select_table,select_cat_factor),
    sort_alpha,
    backend
)

In [ ]:
pl.alpha_plot(
    tables_dict=mgf_parquet_dfs,
    table_name=select_table.value,
    factor=select_cat_factor.value,
    metadata=full_metadata,
    order=sort_alpha.value,
    backend=backend.value,
)

In [ ]:
pl.av_alpha_plot(
    tables_dict=mgf_parquet_dfs,
    table_name=select_table.value,
    factor=select_cat_factor.value,
    metadata=full_metadata,
    order=sort_alpha.value,
    backend=backend.value,
)

## Beta diversity

In [ ]:
pn.Column(
    pn.Row(
        select_table_beta,
        select_taxon,
        select_beta_factor,
        
    ),
    beta_norm
)


In [ ]:
pl.beta_plot(
    tables_dict=mgf_parquet_dfs,
    table_name=select_table_beta.value,
    norm=beta_norm.value,
    taxon=select_taxon.value,
    backend=backend.value,
)

In [ ]:
beta_pc_plot, explained_var_indicator = pl.beta_plot_pc(
        tables_dict=mgf_parquet_dfs,
        metadata=full_metadata,
        table_name=select_table_beta.value,
        factor=select_beta_factor.value,
        taxon=select_taxon.value,
    )
print('Explained variance:', explained_var_indicator)
beta_pc_plot.opts(
    title=f'Beta diversity PCA plot for {select_table_beta.value} table',
    width=1200,
    height=800,
)